In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Characterizing sense vs stop UGA

This notebook contains the code to reproduce the results for _phase 1_ of the Master-thesis

## 0: Implementation of (some of) the used data-structures and algorithms

In [ ]:
### Define the global encoding of the bases and codons as numbers:
base_order = "TCAG"
codon_list = [a + b + c for a in base_order for b in base_order for c in base_order]
rna_codon_list = [c.replace("T", "U") for c in codon_list]

def codon2index(codon):
    result = base_order.index(codon[0]) * 16
    result += base_order.index(codon[1]) * 4
    result += base_order.index(codon[2])
    return result


### Simple approximation based on marginals
def fit_codon_usage(counts):
    """
    Produces an independent codon approximation based on the given counts: This means that three categoricals 
    over the nucleobases, one for each position in a codon will be inferred (as the marginals of the empirical codon usage)
    counts: The counts of the codons (unnormalised empirical), a 64-entry array
    """
    positional_fs = np.zeros((3, 4))
    for c in codon_list:
        for i in range(3): # position in codon
            positional_fs[i, base_order.index(c[i])] += counts[codon2index(c)]

    independent_codon_approximation = np.zeros(64)
    for c in codon_list:
        independent_codon_approximation[codon2index(c)] = np.product([positional_fs[i, base_order.index(c[i])] 
                                                                      for i in range(3)])
    
    return independent_codon_approximation

### Classes and functions for handling the raw data
class Transcript:
    
    def __init__(self, head, sequence=""):
        self.head = head
        self.sequence = sequence
        self.hits = []
        self.stop_codon = "TGA"
        
    def has_hits(self, count_reverse_Complement=False):
        return len([h for h in self.hits if h.frame >= 0]) > 0
        
    def closest_to_c_terminus(self):
        if not self.has_hits():
            return None, None
        hit_index = np.argmax([h.end_index for h in self.hits if h.frame >= 0]) # exclude rev'comp
        return hit_index, self.hits[hit_index].end_index
        
    def find_first_downstream_stop(self, starting_from):
        # Index suffices to imply frame
        i = starting_from
        while i < len(self.sequence) - 3:
            if self.sequence[i:i+3] == self.stop_codon:
                return i
            i += 3
            
    def find_c_terminal_stop_based_on_hits(self, also_upstream=True):
        if not self.has_hits():
            return None
        hit_i, sequence_i = self.closest_to_c_terminus()
        # Preferred:
        downstream = self.find_first_downstream_stop(sequence_i)
        if downstream is not None:
            return downstream
        elif also_upstream:
            i = sequence_i-3
            while i > 0:
                if self.sequence[i:i+3] == self.stop_codon:
                    return i
                i -= 3
        
        return None
                
    def utr_length(self, require_top_hits=False):
        """
        uses find_c_terminal_stop_based_on_hits
        """
        stop = self.find_c_terminal_stop_based_on_hits()
        if stop is None:
            return "no stop"
        if require_top_hits and not self.are_top_hits_close_to_stop(stop):
            return "not enough hits"
        
        i = len(self.sequence) - 7
        while i > stop:
            if "AAAAAAA" not in self.sequence[i:i+7]:
                return i - stop - 2 # -2 because stop is U's index in UGA
            i -= 1
    
    def are_top_hits_close_to_stop(self, stop_index, tolerance=6, top_n=10, how_many=None):
        """
        stop_index: index of the stop-codon's first base
        tolerance: how many amino-acids (!) off may the top matches be?
        top_n: Which top matches have to be close? Default: top ten
        how_many: how many of the top_n have to be close? 
        -> top_n=10, how_many=3 can mean that the 8th, 9th and 10th best matches are close
         if how_many=None (default), either all top_n, or all the hits (whichever are fewer) have to be close
        """
        tolerance_in_nucleotides = 3 * tolerance 
        if not self.has_hits() or stop_index is None:
            return False
        # Unnecessary due to ordering of the hits in Blast-output, but still done for robustness
        score_sorted_hits = sorted([h for h in self.hits if h.frame >= 0], 
                                   key=lambda h: h.score, reverse=True)
        
        close = 0 
        for hit in score_sorted_hits[:np.min((top_n, len(score_sorted_hits)))]:
            if np.absolute(hit.end_index - stop_index) <= tolerance:
                close += 1
        if how_many is None:
            return close == np.min((top_n, len(score_sorted_hits)))
        return close >= how_many
                                             
    def list_codon_occurrences(self, codon, origin, relative=True):
        postprocess_index = lambda j: j
        if relative:
            postprocess_index = lambda j: j - origin
        occurrences = []
        
        i = origin - 3
        while i > 0:
            if self.sequence[i:i+3] == codon:
                occurrences.append(postprocess_index(i))
            i -= 3
            
        i = origin
        while i < len(self.sequence) - 3:
            if self.sequence[i:i+3] == codon:
                occurrences.append(postprocess_index(i))
            i += 3
        return occurrences
    
    def list_inframe_stops(self, main_stop=None, relative=True, include_UAR=False, include_reference=None):
        """
        Produces a list of indices (always on first base of codon) where UGA=TGA is found in-frame 
        starting from the main_stop codon.
        @param relative: (not yet implemented) if true, should return actual sequence-index, never < 0
        @param include_UAR: (not yet implemented) whether to also count UAA/UAG as stops (not in Loxodes)
        """
        if not self.has_hits():
            return []
        
        if main_stop is None:
            main_stop = self.find_c_terminal_stop_based_on_hits()
            if main_stop is None: # Peculiar case
                print("%s: No stop could be found based on the %s hits" % (self.head, len(self.hits)))
                return []
        
        stops = self.list_codon_occurrences("TGA", main_stop, relative=relative)
        
        if include_UAR:
            uaas = self.list_codon_occurrences("TAA", main_stop, relative=relative)
            uags = self.list_codon_occurrences("TAG", main_stop, relative=relative)
            
            return stops, uaas, uags
        
        return stops
    
    def codon_counts(self):
        # +1 means the first codon is 0:3, +2 means 1:4, +3 means 2:5
        # Based on https://www.ncbi.nlm.nih.gov/Taxonomy/Utils/wprintgc.cgi?chapter=cgencodes
        # Use the sequence TCAG as default: Yield 4^3=64-entry count-vector
        # Go by hits
        counts = np.zeros(64)
        for hit in self.hits:
            if hit.frame < 0:
                continue
            else:
                i = hit.start_index - 1 # First base has ordinal 1. = index 0
                while i < hit.end_index - 3:
                    counts[codon2index(self.sequence[i:i+3])] += 1
                    i += 3
        return counts
    
    def base_frequencies(self):
        """
        computes the base frequencies in the matching region (of the best match) and in the 3'UTR and returns both:
        matching region = from start of top match to putative stop (excluding the stop)
        3' UTR = from the stop to the 5' end of the poly-A (excluding polyA)
        """
        frequ_match = np.zeros(4)
        frequ_utr = np.zeros(4)
        # Unnecessary due to ordering of the hits in Blast-output
        score_sorted_hits = sorted([h for h in self.hits if h.frame >= 0], 
                                   key=lambda h: h.score, reverse=True)
        match_start = score_sorted_hits[0].start_index - 1 # blast gives ordinals, not indices
        stop = self.find_c_terminal_stop_based_on_hits() # this is an index
        for i in range(match_start, stop):
            frequ_match[base_order.index(self.sequence[i])] += 1
        
        i = len(self.sequence) - 7
        # Move backwards through the Poly-A-tail
        while i >= stop:
            if "AAAAAAA" not in self.sequence[i:i+7]:
                break
            i -= 1
        
        # Tail has ended, now collect base frequencies
        while i >= stop:
            frequ_utr[base_order.index(self.sequence[i])] += 1
            i-=1
        total = frequ_match + frequ_utr
        return frequ_match/np.sum(frequ_match), frequ_utr/np.sum(frequ_utr), total/np.sum(total)
        
    def base_counts(self, upstream=60, downstream=60):
        length = len(self.sequence)
        counts = np.zeros((4, upstream + 3 + downstream))
        stop = self.find_c_terminal_stop_based_on_hits() # this is an index
        for i in range(downstream + 3 + upstream):
            sequ_i = min(max(0, stop + i - upstream), length - 1)
            counts[base_order.index(self.sequence[sequ_i]), i] = 1
        return counts
        
class Hit:
    def __init__(self, name, frame=0, score=0, evalue=0):
        self.start_index = None
        self.end_index = -1 # last match to query (this is already w.r.t. nucleotides)
        self.frame = frame
        self.name = name
        self.score = score
        self.evalue = evalue
        self.startline = -1
        self.endline = -1
    
    def __str__(self):
        return "%s (ll.%s-%s): Score=%s, E=%s, Frame=%s: %s--%s" % (self.name, self.startline, self.endline, 
                                                                    self.score, self.evalue, 
                                                                    self.frame, self.start_index, self.end_index)
    

## 1.1: Read blast-output
BlastX of polyA-tailed transcripts (`polyA-terminated.LmagMAC.fasta`) against ciliates.org-derived protein database

In [ ]:
transcripts = []
current_hit = None
current_hitname = ""
with open('blastx_q.polyA_wrt_ciliates_org.out') as blast_result:
    for row, line in enumerate(blast_result):
        if row % 1e6 == 0:
            print("@", row)
            
        if line.strip() == "":
            continue
            
        if line.startswith("Query"):
            if line.startswith("Query= "):
                if current_hit is not None and len(transcripts) != 0:
                    current_hit.endline = row + 3
                    transcripts[-1].hits.append(current_hit)
                    current_hit = None
                transcripts.append(Transcript(line[7:-1].split(" ")[0]))
            else:
                current_hit.endline = row + 3
                current_hit.end_index = int(line.split(" ")[-1])
                if current_hit.start_index is None:
                    current_hit.start_index = int(line.split("  ")[1])
        
        if line.startswith(">"):
            current_hitname = line[1:].replace("\t", " ")[:-1]
            
        if line.startswith(" Score = "):
            if current_hit is not None:
                transcripts[-1].hits.append(current_hit)
                
            current_hit = Hit(current_hitname)
            current_hit.startline = row
            content = [e.strip() for e in line.split(",")]
            current_hit.score = float(content[0].split(" ")[2])
            current_hit.evalue = float(content[1].split(" ")[2])
        
        if line.startswith(" Frame = "):
            current_hit.frame = int(line[8:])               
        
            
print("Found", len(transcripts), "transcripts")

## 1.2: Read nucleotide sequences

In [ ]:
transcript_dict = {t.head:t for t in transcripts}
current = None
with open('polyA-terminated.LmagMAC.fasta') as trinity:
    for row, line in enumerate(trinity):
        if line.startswith(">"):
            current = transcript_dict[line[1:-1].split(" ")[0]]
        else:
            current.sequence = line[:-1]

# 2 Characterise sense- vs stop-UGA in _Loxodes magnus_
### Collect the occurrences of UGA in the transcripts for which the top hits are close to the putative Stop

In [ ]:
# indices[STOP][other]
stop_codons= ["TGA", "TAA", "TAG"]
indices = {"TGA": {"TGA": [], "TAA": [], "TAG": []}, 
           "TAA": {"TGA": [], "TAA": [], "TAG": []}, 
           "TAG": {"TGA": [], "TAA": [], "TAG": []}}
useful_transcripts = {c:[] for c in stop_codons}
required_top = {"TGA": 4, "TAA": 3, "TAG": 3}

scores = []
how_many = {c:0 for c in stop_codons}
for i in range(len(transcripts)):
    for stop in stop_codons:
        transcripts[i].stop_codon = stop
        stop_index = transcripts[i].find_c_terminal_stop_based_on_hits(also_upstream=True)
        if transcripts[i].are_top_hits_close_to_stop(stop_index, top_n = 10, how_many=required_top[stop]):
            how_many[stop] += 1
            scores += [h.score for h in transcripts[i].hits]
            useful_transcripts[stop].append(transcripts[i])
            uga, uaa, uag = transcripts[i].list_inframe_stops(main_stop=stop_index, include_UAR=True)
            indices[stop]["TGA"] += uga
            indices[stop]["TAA"] += uaa
            indices[stop]["TAG"] += uag
    
    transcripts[i].stop_codon = "TGA" # restore default

print("%s of the transcripts fulfilled the criterion" % how_many)
print("Their scores range from %s to %s, mean: %s/median: %s" % (np.min(scores), np.max(scores), 
                                                                 np.mean(scores), np.median(scores)))

### Plot occurrences of UGA/UAA/UAG relative to putative stops

In [ ]:
upstream = 70; downstream = 20
filter_start, filter_end = (-3 * upstream, 3 * downstream)

fig, axes = plt.subplots(3, 3, figsize=(12, 8)) # , sharey=True)
plt.rc('font', size=8) 
plt.tight_layout(pad=0, h_pad=4, w_pad=4)

# /3 to get from nucleotide-index to amino-acids/codons
for stop_i in range(len(stop_codons)):
    for ref_i in range(len(stop_codons)):
        filtered = [i/3 for i in indices[stop_codons[stop_i]][stop_codons[ref_i]] if filter_start <= i <= filter_end]
        plot_data = np.zeros(upstream + 1 + downstream) # #codons
        for i in filtered:
            plot_data[int(i) + upstream] += 1
        
        # axes[stop_i, ref_i].hist(filtered, bins=60, log=True)
        # axes[stop_i, ref_i].axvline(0, color="black", linewidth=1, alpha=1)
        axes[stop_i, ref_i].bar(range(-upstream, downstream+1), plot_data, log=True, color="tab:blue")
        axes[stop_i, ref_i].set_title("absolute prevalence of %s w.r.t Stop-%s" % (stop_codons[ref_i], stop_codons[stop_i]))
        axes[stop_i, ref_i].set_xlabel("#codons downstream of Stop-%s" % stop_codons[stop_i])
        axes[stop_i, ref_i].set_ylabel("#%s in the transcripts as that codon" % stop_codons[ref_i])

fig.savefig('out/stop_codons_before_stop_codons_all_UAA.UGA.UAG.png', bbox_inches='tight')
plt.show()

For plotting in LaTeX: Write part of this into a csv-file

In [ ]:
upstream = 70; downstream = 20
filter_start, filter_end = (-3 * upstream, 3 * downstream)

# /3 to get from nucleotide-index to amino-acids/codons
filtered = [i/3 for i in indices["TGA"]["TGA"] if filter_start <= i <= filter_end]
uga_data = np.zeros(upstream + 1 + downstream) # #codons
for i in filtered:
    uga_data[int(i) + upstream] += 1
    
filtered = [i/3 for i in indices["TGA"]["TAA"] if filter_start <= i <= filter_end]
uaa_data = np.zeros(upstream + 1 + downstream) # #codons
for i in filtered:
    uaa_data[int(i) + upstream] += 1

with open("out/depletion_before_uga.csv", "w") as out:
    out.write("codon,uga,uaa\n") # could actually just write all the codons into this file as columns
    for i in range(-upstream, downstream+1):
        out.write("%s,%s,%s\n" % (i, uga_data[i+upstream], uaa_data[i+upstream]))

### Compare codon frequencies around UGA-stop
Is there a general depletion of some codons (like for UAA), or is this a special feature of UAA?

From now on, no longer need the other 'stops'

In [ ]:
useful_transcripts = useful_transcripts["TGA"]

In [ ]:
upstream = 70; downstream = 20
filter_start, filter_end = (-3 * upstream, 3 * downstream)

# This codon offset serves to define where in the codon-list the plotting shall start 
# (as only 16 of the 64 codons will be plotted at a time)
codon_offset = 0 # 16, 32, 48
file_format = "png"
nrows = 4; ncols = 4

fig, axes = plt.subplots(nrows, ncols, figsize=(10, 8), sharey=True)
plt.rc('font', size=8) 
plt.tight_layout(pad=0, h_pad=4, w_pad=4)

for row in range(nrows):
    for col in range(ncols):
        indices = []
        for i in range(len(useful_transcripts)):
            stop_index = useful_transcripts[i].find_c_terminal_stop_based_on_hits(also_upstream=True)
            indices += useful_transcripts[i].list_codon_occurrences(codon_list[codon_offset + col + row*ncols], 
                                                                    stop_index, relative=True)
        
        filtered = [i/3 for i in indices if filter_start <= i <= filter_end]
        plot_data = np.zeros(upstream + 1 + downstream) # #codons
        for i in filtered:
            plot_data[int(i) + upstream] += 1
        
        axes[row, col].bar(range(-upstream, downstream+1), plot_data, log=True, color="tab:blue")
        axes[row, col].set_title("%s" % codon_list[codon_offset + col + row*ncols])
        axes[row, col].set_xlabel("#nt from stop-UGA")
        axes[row, col].set_ylabel("#%s" % codon_list[codon_offset + col + row*ncols])

fig.savefig('out/codon_usage_%s2%s.%s' % (codon_list[codon_offset], codon_list[codon_offset + nrows * ncols - 1], file_format), 
            bbox_inches='tight')
plt.show()
#

### Assess 3'-UTR-length
3'-UTR starts at the found STOP, and ends at=before first consecutive seven A
TGA|*UTR*|AAAAAAA

Note! This is for a smaller subset of more reliable transcripts

In [ ]:
lengths = []
total_n = 0
n_with_any_hits = 0
n_no_stop = 0
n_not_enough = 0
small_n = 0
for transcript in transcripts:
    total_n += 1
    if transcript.has_hits():
        n_with_any_hits += 1
        length = transcript.utr_length(True)
        # if length is not None:
        if length not in ["no stop", "not enough hits"]:
            lengths.append(length)    
            small_n += 1
        elif length == "no stop":
            n_no_stop += 1
        elif length == "not enough hits":
            n_not_enough += 1
            
print("of all \t%s transcripts\n\t%s had any hits\n\t%s thereof were used, while %s had no stop and %s had not enough hits"\
      % (total_n, n_with_any_hits, small_n, n_no_stop, n_not_enough))

In [ ]:
print("%s of the %s transcripts have a 3'UTR of length 0" % (len([0 for l in lengths if l == 0]), len(lengths)))
print("3'UTR-length:\tmedian=%s\tmean=%s\trange: [%s, %s]" % (np.median(lengths), np.mean(lengths), 
                                                              np.min(lengths), np.max(lengths)))
plt.hist(np.log10(lengths), bins=20, edgecolor="k", color="w", linewidth=0.5, align='left')
plt.axvline(np.log10(np.median(lengths)), color="r")
plt.axvline(np.log10(np.mean(lengths)), linestyle="dashed", linewidth=0.5)
plt.xticks([0, 1, 2, 3], ["1", "10", "100", "1000"])
plt.xlabel("UTR-length")
plt.ylabel("#transcripts with this UTR-length")
plt.show()

# for latex:
hist, bin_edges = np.histogram(np.log10(lengths), bins=20)
bar_width = [bin_edges[i+1] - bin_edges[i] for i in range(len(hist))][0]
print("log(median)=", np.log10(np.median(lengths)), "log(mean)=", np.log10(np.mean(lengths)))
print("\nxlower,count")
print("\n".join(["%s,%s" % (l,c) for (c, l) in zip(hist, bin_edges[:-1])]))

### Assess codon and nucleotide biases:

Average base frequencies (incl GC-content) for the top-matching region (~coding region) vs 3'-UTR:

In [ ]:
matching_frequencies = np.zeros(4)
utr_frequencies = np.zeros(4)
total_frequencies = np.zeros(4)

upstream = 72; downstream = 30
counts = np.zeros((4, upstream + 3 + downstream))
n = 0
for ut in useful_transcripts:
    n += 1
    m_f, u_f, t_f = ut.base_frequencies()
    matching_frequencies += m_f
    utr_frequencies += u_f
    total_frequencies += t_f
    
    counts += ut.base_counts(upstream, downstream)

counts = counts / n
    
matching_frequencies = matching_frequencies/len(useful_transcripts)
utr_frequencies = utr_frequencies/len(useful_transcripts)
total_frequencies = total_frequencies/len(useful_transcripts)

print("\t\tT\tC\tA\tG\t|\tGC")
print("Matching:\t%s\t|\t%s" % ("\t".join([str(np.round(f, 3)) for f in matching_frequencies]), 
                                matching_frequencies[1] + matching_frequencies[3]))
print("3'-UTR:\t\t%s\t|\t%s" % ("\t".join([str(np.round(f, 3)) for f in utr_frequencies]), 
                             utr_frequencies[1] + utr_frequencies[3]))
print("total:\t\t%s\t|\t%s" % ("\t".join([str(np.round(f, 3)) for f in total_frequencies]), 
                             total_frequencies[1] + total_frequencies[3]))

plt.plot(range(-upstream, 3+downstream), counts[0,], label="U")
plt.plot(range(-upstream, 3+downstream), counts[1,], label="C")
plt.plot(range(-upstream, 3+downstream), counts[2,], label="A")
plt.plot(range(-upstream, 3+downstream), counts[3,], label="G")
plt.xlabel("#nucleotides downstream of stop-UGA's U")
plt.ylabel("base-frequency")
plt.legend()
plt.show()

with open("out/base_frequencies.csv", "w") as out:
    out.write("nt,u,c,a,g\n")
    for i in range(-upstream, 3+downstream):
        out.write("%s,%s,%s,%s,%s\n" % (i, counts[0,i+upstream], counts[1,i+upstream], 
                                      counts[2,i+upstream], counts[3,i+upstream]))

## Collect codon-usage in the matching regions

Note: setting `recompute = True` will then trigger a rather slow computation, which is why the option to abbreviate this exists

In [ ]:
recompute = False
# This was computed in a previous run (to save time):
counts = np.array([0.035354874328046015, 0.014842373647985943, 0.032875613243826006, 0.014088955968809047, 
                   0.01900080444301719, 0.00356805161649493, 0.016410789395218176, 0.003566107265019291, 
                   0.03079942547932245, 0.010243065785261898, 0.03256178565684137, 0.01396590629685077, 
                   0.011364882516173022, 0.009150627279266018, 0.008338184644578662, 0.010085452951120004, 
                   0.020381914331429562, 0.0037948926219861117, 0.014436013448394005, 0.004193003216031411, 
                   0.015471778738283292, 0.001843448892869604, 0.010669637981383926, 0.0008325064901526376, 
                   0.016098859865626425, 0.004345440371721485, 0.012871421592397193, 0.00419746596560883, 
                   0.0026233930814641418, 0.0008647086541158347, 0.002690945407017761, 0.00021116582907091368, 
                   0.0390077552217781, 0.011168493758316963, 0.031892650984725136, 0.019177258968839674, 
                   0.023849905917461504, 0.003764495927250293, 0.015866139511625573, 0.00358642110853144, 
                   0.042859895106496945, 0.017490700722421013, 0.055141659708048775, 0.024078061674902878, 
                   0.017581761183196758, 0.00807422504353588, 0.024174140414249083, 0.004461342237302653, 
                   0.021964607173782912, 0.004501127372020842, 0.019434348528001865, 0.005903606609032139, 
                   0.024355159536631046, 0.003895461887359386, 0.016787743593445217, 0.002529610528622502, 
                   0.04129483105085605, 0.011945771407744912, 0.046644269712881506, 0.014030486542291627, 
                   0.015584338171089672, 0.005736170170529994, 0.03738262922317307, 0.004091433998470666])


if recompute:
    counts = np.zeros(64)
    for i in range(len(transcripts)):
        if i % 1e2 == 0:
            print("@", i, end=" - ")
        if i % 1e3 == 900:
            print("|")
        counts += transcripts[i].codon_counts()


    counts = counts/np.sum(counts)
    
print("\n%s" % (counts * 64)) # > 1 means overrepresented compared to naive Null-model (uniform)
print(counts[codon2index("TGA")])

# UGA is underrepresented w.r.t. uniform 0-model; but not remarkably so
print("\nUGA-usage:", counts[codon2index("TGA")] * 64) 
print("UGG-usage:", counts[codon2index("TGG")] * 64) 
# this is the other Typtophane-codon: it is also slightly underrepresented

# Marginalise to get base-usage for codon-pos 0, 1, 2
# then remultiply (independency-assumption) and compare
independent_codon_approximation = fit_codon_usage(counts)

plt.figure(figsize=(14, 4))
w = 0.3
plt.bar(np.arange(64) - w/2, 64*counts, width=w)
plt.bar(np.arange(64) + w/2, 64*independent_codon_approximation, width=w)
plt.xticks(range(64), codon_list, rotation=90)
plt.axhline(1, color="k", linewidth=0.5)

plt.show()
# W-codons are TGA and TGG
# AAA-usage is suspiciously high (-> polyA? -- i rely on the hits to stop before polyA)

- Blue: empirical codon usage relative to uniform distribution
- Orange: independent-base-wise approximation relative to uniform distribution

The horizontal line marks exact accord with a uniform distribution. Any bar above it means the codon is overrepresented, any below means underrepresented. Contrast the good agreement of the two models e.g. for ATT as opposed to the strong disagreement for GGA and CTT (the latter over- vs underrepresented)

In [ ]:
if recompute:
    print(", ".join([str(c) for c in counts])) # copy this into above cell if needed

# Position Count/Probability/Weight Matrices
Further of interest: base and codon use left and right of UGA (stop vs sense)

Include pseudocounts outside the stop-UGA

In [ ]:
upstream = 21; downstream = 12 # 3
pattern_length = upstream + 3 + downstream

count_matrix_stop = np.ones((4, pattern_length)) # codon to the left, Stop, codon to the right
count_matrix_sense = np.ones((4, pattern_length))
count_matrix_utr = np.ones((4, pattern_length))

"""
Without any pseudocounts:
count_matrix_stop = np.zeros((4, pattern_length)) 
count_matrix_sense = np.zeros((4, pattern_length))
count_matrix_utr = np.zeros((4, pattern_length))
"""

count_matrix_stop[:,upstream:upstream+3] = 0 # stop-UGA: no pseudocounts
count_matrix_sense[:,upstream:upstream+3] = 0 # stop-UGA: no pseudocounts
count_matrix_utr[:,upstream:upstream+3] = 0 # stop-UGA: no pseudocounts


for transcript in useful_transcripts:
    main_stop = transcript.find_c_terminal_stop_based_on_hits()
    shifted = False
    if transcript.sequence[main_stop-6:main_stop-3] == "TGA":
        main_stop -= 6
        shifted = True
    elif transcript.sequence[main_stop-9:main_stop-6] == "TGA":
        main_stop -= 9
        shifted = True
    elif transcript.sequence[main_stop-12:main_stop-9] == "TGA":
        main_stop -= 12
        shifted = True
    while transcript.sequence[main_stop-3:main_stop] == "TGA":
        main_stop -= 3
        shifted = True
    
    if shifted:
        print(transcript.head, "got shifted!")
    
    
    UGA_occurrences = transcript.list_inframe_stops(main_stop=main_stop, relative=False)
    for occurrence in UGA_occurrences:
        relevant = None
        if occurrence == main_stop:
            relevant = count_matrix_stop
        elif occurrence < main_stop:
            relevant = count_matrix_sense
        else:
            relevant = count_matrix_utr        
        
        for j in range(pattern_length):
            if occurrence + j - upstream < len(transcript.sequence):
                relevant[base_order.index(transcript.sequence[occurrence + j - upstream]), j] += 1
    
print("Sense:", np.sum(count_matrix_sense, axis=0)[0])
print("STOPs:", np.sum(count_matrix_stop, axis=0)[0])
print("3'UTR:", np.sum(count_matrix_utr, axis=0)[0])
    
probability_matrix_stop = count_matrix_stop / np.sum(count_matrix_stop, axis=0) # same #sequences for all columns
probability_matrix_sense = count_matrix_sense / np.sum(count_matrix_sense, axis=0)
probability_matrix_utr = count_matrix_utr / np.sum(count_matrix_utr, axis=0)

In [ ]:
print("PPM around sense stop")
print(np.round(probability_matrix_sense, 3)) 
print(np.sum(probability_matrix_sense, axis=0))
print("\nPPM around full stop")
print(np.round(probability_matrix_stop, 3))
print(np.sum(probability_matrix_stop, axis=0))
print("\nPPM around utr stop")
print(np.round(probability_matrix_utr, 3)) 
print(np.sum(probability_matrix_utr, axis=0))

#### Transformation into a Position Weight Matrix (PWM):
Use a position-dependent background model to account for different base composition of UTR and ~coding region. Use the empirical base frequencies computed above

In [ ]:
from matplotlib import colors
file_format = "pdf"
# matching_frequencies and utr_frequencies
PWM_sense = np.log2(probability_matrix_sense / np.tile(matching_frequencies, pattern_length).reshape(pattern_length, 4).T)
PWM_utr = np.log2(probability_matrix_utr / np.tile(utr_frequencies, pattern_length).reshape(pattern_length, 4).T)
PWM_stop = np.log2(probability_matrix_stop / np.append(np.tile(matching_frequencies, upstream), 
                                                       np.tile(utr_frequencies, 3 + downstream)).reshape(pattern_length, 4).T)

pwms = [PWM_sense, PWM_stop, PWM_utr]
xticks = [i for i in range(upstream + 3 + downstream) if (i - upstream) % 3 == 1]
ylabels = ["sense", "stop", "utr"]


fig, axes = plt.subplots(3)

for a in range(len(axes)):
    im = axes[a].imshow(pwms[a], vmin=-1.75, vmax=3.562, cmap="coolwarm", 
                        norm=colors.TwoSlopeNorm(vmin=-1.75, vcenter=0., vmax=3.562))
    axes[a].set_xticks(xticks)
    axes[a].set_xticklabels(np.array((np.array(xticks) - upstream - 1)/3, dtype=int))
    
    axes[a].set_yticks([1.5])
    axes[a].set_yticklabels([ylabels[a]])

plt.tight_layout(h_pad = -15)
fig.colorbar(im, ax=axes, shrink=0.5)
    
fig.savefig('out/PWMs.%s' % file_format, bbox_inches='tight')
plt.show()

print(upstream+downstream+3)

print("PWM around sense UGA")
print(np.round(PWM_sense, 3))
print("\nPWM around stop UGA")
print(np.round(PWM_stop, 3))
print("\nPWM around 3'UTR UGA")
print(np.round(PWM_utr, 3))

print("Range of sense-PWM: [%s, %s]" % (np.min(PWM_sense[:,:upstream]), np.max(PWM_sense)))
print("Range of stop-PWM: [%s, %s]" % (np.min(PWM_stop[:,:upstream]), np.max(PWM_stop)))
print("Range of UTR-PWM: [%s, %s]" % (np.min(PWM_utr[:,upstream+3:]), np.max(PWM_utr)))

print("without respect for the -inf-entries at the UGA of course")

#### Comparing PPMs (probability matrices) according to Aerts, Loo, Thijs, DeMoor, Moreau (2003): Use symmetrized KL:
$c_{1,2} := \frac{1}{w}\sum_{j=1}^{w}\sum_{b\in\{T,C,A,G\}}\Theta_{1}(j,b)\log_2\frac{\Theta_1(j,b)}{\Theta_2(j,b)}$

In [ ]:
def KL(ppm1, ppm2):
    # indices j are here already known to be 0,1,2,6,7,8; w=6
    indices = [i for i in range(upstream + 3 + downstream) if i < upstream or i >= upstream + 3]
    result = 0
    for j in indices:
        for b in range(4):
            result += ppm1[b, j] * np.log(ppm1[b, j] / ppm2[b, j])
    return result / len(indices)

def symm_KL(M, N):
    return 0.5*(KL(M, N) + KL(N, M))

print("distance sense<->stop\t", symm_KL(probability_matrix_sense, probability_matrix_stop))
print("distance sense<->utr\t", symm_KL(probability_matrix_sense, probability_matrix_utr))
print("distance stop<->utr\t", symm_KL(probability_matrix_stop, probability_matrix_utr))

Aerts et al. set 'high-stringency' threshold at 0.2 (for 'same'): these here can be assumed to be same.

# 3 Comparison to other ciliates

In [ ]:
complement = {"T":"A", "C":"G", "A":"T", "G":"C", "N":"N"}

def reverse_complement(sequence):
    return "".join([complement[b] for b in sequence[::-1]])

class Gene:
    def __init__(self, start, end, strand):
        self.start = start
        self.end = end
        self.forward = strand == "+"
        self.validated = False # use this to check for CDS
        
    def feasible(self, sequence, upstream, downstream):
        if self.forward:                
            return self.end-3-3*upstream >= 0 and self.end + 3*downstream < len(sequence)
        else:
            return self.start-1-3*downstream >= 0 and self.start + 2 + 3*upstream < len(sequence)
    
    def count_codons(self, sequence, upstream = 60, downstream=20):
        """ upstream and downstream are in codons (not nt) """
        if self.forward:                
            relevant = sequence[self.end-3-3*upstream : self.end + 3*downstream]
        else:
            relevant = sequence[self.start-1-3*downstream : self.start + 2 + 3*upstream]
            relevant = reverse_complement(relevant)
        # print(relevant[:upstream*3], relevant[3*upstream:3*upstream+3], relevant[3*upstream+3:])
        count = np.zeros((upstream + 1 + downstream, 64))
        for i in range(upstream + downstream + 1):
            if "N" not in relevant[3*i:3*i+3]:
                count[i, codon2index(relevant[3*i:3*i+3])] += 1
            else:
                print("! ENCOUNTERED N !")
        return count

### Global parameters: Upstream and downstream windows (in codons)

In [ ]:
up = 60
down = 20

### Parsing data-files:

In [ ]:
### Parsing of GFF-file:
### (uses up and down)
def parse_gff(file, work_dict, validator="CDS"):
    with open(file) as annotation:
        for line in annotation:
            if "gene" in line:
                contents = line.split("\t")
                if contents[6] not in "+-":
                    # The Euplotes annotation seems not very good
                    continue
                
                if contents[0] not in work_dict:
                    work_dict[contents[0]] = [Gene(int(contents[3]), int(contents[4]), contents[6])]
                work_dict[contents[0]].append(Gene(int(contents[3]), int(contents[4]), contents[6]))

    with open(file) as annotation:
        for line in annotation:
            if validator in line:
                contents = line.split("\t")
                strand = contents[6] == "+"
                start = int(contents[3])
                end = int(contents[4])
                if contents[0] in work_dict:
                    for g in [g for g in work_dict[contents[0]] if g.forward == strand and (g.start == start or g.end == end)]:
                        if (strand and end == g.end and start <= g.end - 3*(1+up)) or (not strand and start == g.start and end >= g.start + 2 + 3*up):
                            g.validated = True
                            
### Parsing of fasta under given annotation
def apply_gff(fasta, genes):
    g_c = 0
    codon_counts = np.zeros((up + 1 + down, 64))
    with open(fasta) as assembly:
        current_sequence = ""
        current_header = None
        for line in assembly:
            if line.startswith(">"):
                if current_header is not None and current_header in genes:
                    print(current_header)
                    for g in genes[current_header]:
                        if g.validated and g.feasible(current_sequence, up, down):
                            g_c += 1
                            # print("\t%s%s%s" % (g.start, ">" if g.forward else "<", g.end))
                            print("%s gene %s%s%s (len: %s)" %(g_c,g.start, "+" if g.forward else "-", g.end, len(current_sequence)))
                            codon_counts += g.count_codons(current_sequence, up, down)
                            print("\tTAA: %s\tTAG: %s\tTGA: %s" % (codon_counts[up, codon2index("TAA")],
                                                                   codon_counts[up, codon2index("TAG")],
                                                                   codon_counts[up, codon2index("TGA")]))
                        else:
                            if g.validated:
                                print("\ntoo close to edges\n")
                            else:
                                print("\n-> No sufficient CDS found\n")
                current_header = line[1:-1]
                current_sequence = ""
            else:
                current_sequence += line[:-1]
    return codon_counts

### 3.1 _Tetrahymena thermophila_

In [ ]:
genes_tetra = {} # contig:[genes] pairs
parse_gff("ciliates/tetrahymena_annotation.gff3", genes_tetra, validator="CDS")
codon_counts_tetra = apply_gff("ciliates/tetrahymena_assembly.fasta", genes_tetra)

### 3.2 _Stentor coeruleus_
Uses the standard genetic code, so do not expect UGA/UAA-depletion

In [ ]:
genes_stentor = {} # contig:[genes] pairs
parse_gff("ciliates/S_coeruleus_Nov2016.gff3", genes_stentor, validator="CDS")
codon_counts_stentor = apply_gff("ciliates/S_coeruleus_Nov2016_assembly.fasta", genes_stentor)

In [ ]:
# Normalising column-wise (per position)
normalised_stentor = codon_counts_stentor.T / np.sum(codon_counts_stentor, axis=1)
normalised_tetra = codon_counts_tetra.T / np.sum(codon_counts_tetra, axis=1)
# [codon, pos]
print(normalised_stentor.shape)
print(np.sum(normalised_stentor, axis=0)) # summing over all codons at a certain position

In [ ]:
xs = range(-up, 1+down)
highlight_stop = ["tab:orange" if x == 0 else "tab:blue" for x in xs]

plt.subplot(2,3,1)
plt.title("UGA")
plt.ylabel("Tetrahymena")
plt.bar(xs, normalised_tetra[codon2index("TGA")], color=highlight_stop, width=1) # this should be the only stop
plt.ylim(0, 0.05)

plt.subplot(2,3,2)
plt.title("UAA")
plt.bar(xs, normalised_tetra[codon2index("TAA")], color=highlight_stop, width=1)

plt.subplot(2,3,3)
plt.title("UAG")
plt.bar(xs, normalised_tetra[codon2index("TAG")], color=highlight_stop, width=1)


plt.subplot(2,3,4)
plt.title("UGA")
plt.ylabel("Stentor")
plt.bar(xs, normalised_stentor[codon2index("TGA")], color=highlight_stop, width=1)

plt.subplot(2,3,5)
plt.title("UAA")
plt.bar(xs, normalised_stentor[codon2index("TAA")], color=highlight_stop, width=1)
plt.xlabel("#codons downstream of stop")

plt.subplot(2,3,6)
plt.title("UAG")
plt.bar(xs, normalised_stentor[codon2index("TAG")], color=highlight_stop, width=1)

plt.tight_layout(pad=3, h_pad=2, w_pad=1)
plt.show()

# 4 Conserved _Tetrahymena_ proteins

In [ ]:
regions_of_interest = {} # contig:[(hit, qstart, qend, sstart, send)]
with open("ciliates/blastx_q.transcriptome_wrt_conserved_tetrahymena.out") as blast:
    for line in blast:
        qseqid, sseqid, pident, length, mismatch, gapopen, qstart, qend, sstart, send, evalue, bitscore = line.split("\t")
        qstart, qend, sstart, send = [int(i) for i in [qstart, qend, sstart, send]]
        if qseqid in regions_of_interest:
            regions_of_interest[qseqid] += [(sseqid, qstart, qend, sstart, send)]
        else:
            regions_of_interest[qseqid] = [(sseqid, qstart, qend, sstart, send)]
        # print(qseqid, sseqid, qstart < qend, sstart < send)

upstream = 72; downstream = 30
periterminal_sequences = []
inframe_stops = []
inframe_taa = []
utr_lengths = []

with open("Lmag_transcriptome.fasta") as transcriptome:
    current_head = None
    current_sequence = ""
    for line in transcriptome:
        if line.startswith(">"):
            if current_head is not None:
                if current_head in regions_of_interest:
                    for g in regions_of_interest[current_head]:
                        hit, qstart, qend, sstart, send = g
                        print(current_head, qstart, qend, hit, sstart, send)
                        polyA_start = len(current_sequence) - 1
                        for i in range(len(current_sequence) - 1, 0, -1):
                            if current_sequence[i] == "A":
                                polyA_start = i
                            else:
                                break
                        stop = -1
                        # here doing -3 because of the below-described case!
                        for i in range(qend-3, len(current_sequence), 3):
                            if current_sequence[i:i+3] == "TGA":
                                stop = i
                                if i-qend <= 90:
                                    periterminal_sequences += [current_sequence[i-upstream:i+downstream+3]]
                                    utr_lengths += [polyA_start - i] # UGA is part of the UTR
                                    
                                    for j in range(i, len(current_sequence), 3):
                                        if current_sequence[j:j+3] == "TGA":
                                            inframe_stops.append(j-i)
                                        elif current_sequence[j:j+3] == "TAA":
                                            inframe_taa.append(j-i)
                                    for j in range(i-3, 0, -3):
                                        if current_sequence[j:j+3] == "TGA":
                                            inframe_stops.append(j-i)
                                            if j-i >= -10:
                                                print("! very close stop", current_head, j-i, "\n")
                                        elif current_sequence[j:j+3] == "TAA":
                                            inframe_taa.append(j-i)
                                break
            current_head = line[1:-1].split(" ")[0]
            current_sequence = ""
        else:
            current_sequence += line[:-1]
            
print("Found %s transcripts with sufficiently high scoring alignment to one of the conserved proteins" % 
      len(periterminal_sequences))

#### There are a few cases, where UGA occurs directly before the putative stop:
All of these match to 60S_ribosomal_protein_L39_chr_024, and can be reasonably moved by one codon.

In [ ]:
print("3'UTR-length:\tmedian=%s\tmean=%s\trange: [%s, %s]" % (np.median(utr_lengths), 
                                                              np.mean(utr_lengths),
                                                              np.min(utr_lengths), 
                                                              np.max(utr_lengths)))

plt.hist(np.log10(utr_lengths), bins=20, edgecolor="k", color="w", linewidth=0.5, align='left')
plt.axvline(np.log10(np.median(utr_lengths)), color="r")
plt.axvline(np.log10(np.mean(utr_lengths)), linestyle="dashed", linewidth=0.5)
plt.xticks([0, 1, 2, 2.681241237375587, 3], ["1", "10", "100", "480", "1000"])
plt.xlabel("UTR-length")
plt.ylabel("#transcripts with this UTR-length")
plt.show()


# for latex:
hist, bin_edges = np.histogram(np.log10(utr_lengths), bins=20)
bar_width = [bin_edges[i+1] - bin_edges[i] for i in range(len(hist))][0]
print(len(hist), bar_width * 2, "log(median)=", np.log10(np.median(utr_lengths)), 
      "log(mean)=", np.log10(np.mean(utr_lengths)))
print("log(20)=", np.log10(20), "log(400)=", np.log10(400))
result = "xlower,count\n"
result += "\n".join(["%s,%s" % (l,c) for (c, l) in zip(hist, bin_edges[:-1])])
with open("out/tetrah_utr_lengths.csv", "w") as out:
    out.write(result)
print(result)

In [ ]:
counts = np.zeros((4, upstream + 3 + downstream))
n = 0
for seq in periterminal_sequences:
    if len(seq) == upstream + 3 + downstream:
        n += 1
        for i in range(upstream + 3 + downstream):
            counts[base_order.index(seq[i]), i] += 1

print(n)
counts = counts/n
    
plt.plot(range(-upstream, 3+downstream), counts[0,], label="U")
plt.plot(range(-upstream, 3+downstream), counts[1,], label="C")
plt.plot(range(-upstream, 3+downstream), counts[2,], label="A")
plt.plot(range(-upstream, 3+downstream), counts[3,], label="G")
plt.xlabel("#nucleotides downstream of stop-UGA's U")
plt.ylabel("base-frequency")
plt.legend()
plt.show()
# print("\n".join(periterminal_sequences))
print("clearly very noisy due to small sample size")

with open("out/tetrah_base_frequencies.csv", "w") as out:
    out.write("nt,u,c,a,g\n")
    for i in range(-upstream, 3+downstream):
        out.write("%s,%s,%s,%s,%s\n" % (i, counts[0,i+upstream], counts[1,i+upstream], 
                                        counts[2,i+upstream], counts[3,i+upstream]))

In [ ]:
upstream_codons = 200
downstream_codons = 20
stop_counts = np.zeros(upstream_codons + 1 + downstream_codons)
taa_counts = np.zeros(upstream_codons + 1 + downstream_codons)
# print(inframe_stops)
for stop in [s for s in inframe_stops if -3*upstream_codons <= s <= 3*downstream_codons]:
    if stop % 3 != 0:
        print("sth went wrong, not divisble by 3")
    stop_counts[int(stop/3) + upstream_codons] += 1
    
for taa in [s for s in inframe_taa if -3*upstream_codons <= s <= 3*downstream_codons]:
    if taa % 3 != 0:
        print("sth went wrong, not divisble by 3")
    taa_counts[int(taa/3) + upstream_codons] += 1
    
print(stop_counts[upstream_codons], taa_counts[upstream_codons])
    
plt.subplot(1,2,1)
plt.bar(range(-upstream_codons, downstream_codons + 1), stop_counts)
plt.xlabel("#codons upstream if inferred stop")
plt.ylabel("#UGA")

plt.subplot(1,2,2)
plt.bar(range(-upstream_codons, downstream_codons + 1), taa_counts)
plt.xlabel("#codons upstream if inferred stop")
plt.ylabel("#UAA")

plt.show()

print("Do not find any UGA upstream in a big window!\nWhile this is not quite the same sight as for the heuristic, there at least is no eclatant contradiction")

with open("out/tetrah_depletion_before_uga.csv", "w") as out:
    out.write("codon,uga,uaa\n")
    for i in range(-upstream_codons, downstream_codons+1):
        out.write("%s,%s,%s\n" % (i, stop_counts[i+upstream_codons], taa_counts[i+upstream_codons]))